# Transport Equation Solver with Fourier Transform

In this notebook, we implement the numerical solver of the 1D transport equation
$\begin{equation}\frac{\partial \rho}{\partial t} + \frac{\partial (\rho v)}{\partial x} = 0 \end{equation} \tag*{}$
 on $(t,x)\in [0,1]\times [0,L]$ for a given $v:[0,1]\times [0,L]\to \mathbb{R} $ using the discrete Fourier transform. In addition, we implement the calculation of the Wasserstein energy consistent with this method. This notebook is available at https://github.com/mao1756/pdot.

In [11]:
# Import relevant libraries

import numpy as np
import matplotlib.pyplot as plt

# Define Constant(s)
L = 1 # The length of the space
N0 = 5 # The number of time steps
N1 = 5 # The number of space steps

## The Staggered Grids
For the discretization of the time-space rectangle $[0,1]\times [0,L]$, we use the staggered grid, which is known to be more stable for fluid dynamical calculations. Let $N_0$ be the number of time steps and $N_1$ be the number of space steps. We introduce the **time staggered grid**
$\begin{equation} \mathcal{G}_s^t = \left\{\left(\frac{i_0}{N_0}, L\frac{i_1+1/2}{N_1}\right)\middle| i_0= 0,\cdots ,N_0, i_1 = 0,\cdots N_1-1 \right\}\end{equation} \tag*{}$ 
, the **space staggered grid**
$\begin{equation} \mathcal{G}_s^x = \left\{\left(\frac{i_0+1/2}{N_0}, L\frac{i_1}{N_1}\right)\middle| i_0= 0,\cdots ,N_0-1, i_1 = 0,\cdots N_1 \right\}\end{equation} \tag*{}$ 
, and **the centered grid**
$\begin{equation} \mathcal{G}_c = \left\{\left(\frac{i_0+1/2}{N_0}, L\frac{i_1+1/2}{N_1}\right)\middle| i_0= 0,\cdots ,N_0-1, i_1 = 0,\cdots N_1-1 \right\}.\end{equation} \tag*{}$

In [14]:
# Define the time staggered trid
i0 = np.arange(0, N0+1)
i1 = np.arange(0, N1)
Gst = np.array([[(i/N0, L*(j+0.5)/N1) for i in i0] for j in i1])
# the space staggered grid
i0 = np.arange(0, N0)
i1 = np.arange(0, N1+1)
Gsx = np.array([[((i+0.5)/N0, L*j/N1) for i in i0] for j in i1])
# the centered grid
i0 = np.arange(0, N0)
i1 = np.arange(0, N1)
Gc = np.array([[((i+0.5)/N0, L*(j+0.5)/N1) for i in i0] for j in i1])

# Functions
We define the following:
- $\rho^{st}$: The discretized mass on the time staggered grid
- $v^{sx}$: The discretized velocity field on the space staggered grid
- $\rho^{c}$ : The discretized mass on the centered grid
- $v^{c}$ : The discretized mass on the centered grid 
